In [14]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [15]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
words = open('../data/names.txt', 'r').read().splitlines()

# vocabulary of characters and mapping to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)



In [17]:
block_size = 3 #how many character are taken to predict next one
def build_dataset(words):
    inputs, labels = [],[]

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            inputs.append(context)
            labels.append(ix)
            context = context[1:] + [ix] #crop and append next character
    inputs = torch.tensor(inputs).to(dev)
    labels = torch.tensor(labels).to(dev)
    return inputs, labels

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))
xtrain, ytrain = build_dataset(words[:n1])
xdev, ydev = build_dataset(words[n1:n2])
xtest, ytest = build_dataset(words[n2:])

In [18]:
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
    

    def __call__(self, x):
        #data -> layer is like data @ layer
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])
    
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        #parameters trained with backpropagation
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        #buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True, unbiased=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta

        if self.training:
            self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
            self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]
    
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []
    

ch_embed_dim = 10
n_hidden = 100

C = torch.randn((vocab_size, ch_embed_dim))
layers = [
    Linear(ch_embed_dim * block_size, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, n_hidden), Tanh(),
    Linear(n_hidden, vocab_size),
]

with torch.no_grad():
    #make last layer less confindent 
    layers[-1].weight *= 0.1
    #for all other layers apply gain
    for layer in layers[:-1]:
        if isinstance(layer, Linear):
            layer.weight *= 5/3
parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(sum(p.nelement() for p in parameters))
for p in parameters: 
    p.requires_grad = True

46497


In [20]:
#optimization
max_steps = 200000
batch_size = 32
lossi = []
for i in range(max_steps):
    #construct batch
    ix = torch.randint(0, xtrain.shape[0], (batch_size,))
    xbatch, ybatch = xtrain[ix], ytrain[ix]

    #forward pass 
    embed = C[xbatch]
    x = embed.view(embed.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, ybatch)

    #backward pass
    for layer in layers:
        layer.out.retain_grad()
    for p in parameters:
        p.grad = None
    loss.backward()

    learn_rate = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data -= learn_rate * p.grad
    
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/ 200000: 3.3070


KeyboardInterrupt: 